# Getting Started on the CISL Cloud

The CISL Cloud is built on a container management platform known as Kubernetes(K8s). RKE2 is used to deploy K8s on servers running AlmaLinux. Microservices are utilized to provide a tool set that streamlines common tasks. Examples of common tasks that are eliminated due to these microservices are creating DNS records, assigning TLS certificates, configuring a reverse proxy, load balancing applications, scaling resources, mounting shared volumes, and providing a self healing environment that is highly available.    

## Documentation

This documentation site is meant to include all information you may need to navigate the CISL cloud and the resources hosted on it. If you find anything that is not clear or missing please submit an issue to our GitHub repository via the open issue link that appears when you hover over the GitHub icon at the top of the page. The Report Issue link in the navbar opens a Jira ticket that also can be used to communicate a request for enhancing content.    

This Getting Started guide will provide a high level introduction in to each resource. More in depth details on each resource can be found on this Documentation page.  

## JupyterHub

JupyterHub is utilized to provide a Virtual Development Environment. 

### Authentication

Authentication to our JupyterHub is currently controlled via GitHub Teams. Information on how to join the GitHub team can be found at this [how-to request access link](../how-to/k8sJH/access).

### Notebook Options

The JupyterHub uses custom notebook images controlled and maintained by the Cloud team. We provide a CPU image and two GPU images that can be selected after logging in. All the images provide the same extensions and are setup to provide a cohesive experience while switching between images. The users home directory, `/home/jovyan` for all users, is persistent and retains anything written to it. The CPU image is a great place to get started.

### CPU Image

#### Git Extension

The best place to start is to get some code in to the notebook in order to run through real life scenarios. Our custom image includes a Git extension that can be used to interact with Git repositories via a UI instead of using only the command line. 

#### Custom Conda Kernels

The custom images all use a python module called `nb_conda_kernels` that allows any conda environment containing `ipykernel` to be registered as a Jupyter kernel automatically. 

By default the CPU image contains 3 prebuilt conda environments that contain a lot of packages typically required to do scientific analysis. These environments are named:

- cisl-cloud-base
- npl-2023b
- r-4.3

More details on what are included in these packages can be found at this [link to the base conda environments documentation](../how-to/k8sJH/base-conda).

#### Dask Gateway

As of right now Dask Gateway is broken. It spins up a cluster, but the distributed Client can't connect to the cluster and times out. We are still investigating this and will update this Cell when it changes. 

### Data Access & File Sharing

#### GLADE

Read Only access is provided to GLADE campaign and collections. This is mounted inside all notebooks launched on our JupyterHub.

#### Stratus

CISL has an object storage platform known as Stratus that can be used to store and read objects. 

### GPU Images

Two different GPU images are offered. One is built around the [Tensorflow](https://www.tensorflow.org/) Python package, the other is built around [PyTorch](https://pytorch.org/). The GPU images have all the same custom configurations as the CPU image. They contain only 1 custom conda environment by default named:

- cisl-gpu-base

It contains a lot of the same base packages from the `cisl-cloud-base` environment, but adds in the required GPU packages. 

### Web Visualizations